In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load and check data

In [ ]:
train = pd.read_csv("/kaggle/input/tweet-data/train.csv")
test_x = pd.read_csv("/kaggle/input/tweet-data/test_x.csv")
train

# Preprocess text
You may consider applying some preprocessing steps first, such as:
- URL data removal;
- Turn to lowercase;
- Remove hashtags start with @;
- Remove special characters (e.g., =!#%@$^&*)
- Lemmatization (convert words to base form): https://www.nltk.org/howto/stem.html 
- ...

In [ ]:
import re

# Example: URL removal
def remove_url(data):
    re1 = r"http://\S+|www\.\S+"
    re2 = r"https://\S+|www\.\S+"
    url_clean= re.compile("(%s|%s)" % (re1, re2))
    data=url_clean.sub(r'',data)
    return data

# !!! YOUR TASK 1 (4 point): add TWO more preprocessing steps from the above list
# you can choose two functions to implement
def to_lowercase(data):
    # your implementation
    
    return data

def remove_hashtag(data):
    # your implementation
    
    return data

def remove_special(data):
    # your implementation
    
    return data

def lemmatize(data):
    # your implementation
    
    return data


print('before removal: ', train.iloc[5325]['text'])

train['text'] = train['text'].apply(remove_url)
test_x['text'] = test_x['text'].apply(remove_url)

print('after removal: ', train.iloc[5325]['text'])


# Classification based on bag-of-word text features


Generating hand-crafted bag-of-word feature vectors:
- Assign a fixed index to each word occurring in any document of the training set (for instance by building a dictionary from words to integer indices). Each word essentially is an one-hot embedding (dimension=vocab size);
- For each document #i, count the number of occurrences of each word w and store it in X[i, j] as the value of feature #j where j is the index of word w in the dictionary. Each document is a summation of one-hot embedding of all words.
- You could filter out words that are rarely or too frequently appeared in documents.

sklearn tutorial: https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [ ]:
# This package generates bag of words feature for each text
from sklearn.feature_extraction.text import CountVectorizer 

# doc for this function: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
vectorizer = CountVectorizer(stop_words='english', max_df=1.0, min_df=1, max_features=10000) 

# use training data to construct vocabulary and turn training text into BOW vectors
train_bow = vectorizer.fit_transform(train['text'])
vocab = vectorizer.vocabulary_.copy()
# You can change the vocab size by tuning "max_df", "min_df" or "max_features" in "CountVectorizer"
print('vocab size: ', len(vocab))  
print (train_bow.shape) # num_train_text * vocab_size

# !!! YOUR TASK 2 (1 point): transform test text into BOW vectors using vectorizer
test_bow = 
print (test_bow.shape) # num_train_text * vocab_size

In [ ]:
# Use basic ML models on BOW features
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

# Split given training data into training and validation set
X_train,X_val,y_train,y_val = train_test_split(train_bow, train['target'], test_size=0.3, random_state=11)

LR = RandomForestClassifier()
LR.fit(X_train.toarray(),y_train)
print('training recall: ', recall_score(y_train, LR.predict(X_train.toarray())))
print('validation recall: ', recall_score(y_val, LR.predict(X_val.toarray())))

# Optional: apply more advanced model selection technique
# e.g., sklearn.model_selection.GridSearchCV: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html



# Classification based on word2vec embedding features

word2vec:
- Skip-grams or CBOW
- low-dimensional feature vectors with semantic information preserved
 
Before deep learning pervades, gensim is a popular text mining package. Here is a tutorial of gensim's word2vec: https://radimrehurek.com/gensim/models/word2vec.html

Now with deep learning, pytorch and tensorflow all has their way to realize word2vec. Tutorial: https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

In [ ]:
# train word embeddings using gensim library

from gensim.models import Word2Vec

train_docs = [train['text'].iloc[i].split(' ') for i in range(len(train['text']))] # gensim only takes list of word list as `sentences`
w2v_model = Word2Vec(sentences=train_docs, vector_size=100, window=5, negative=20, min_count=2, workers=1)
w2v_model.train(train_docs, total_examples=len(train_docs), epochs=10)

In [ ]:
# check the learned word embeddings

# vocabulary
w2v_vocav = w2v_model.wv.index_to_key
print('example words in vocab: ', [w2v_vocav[i] for i in range(200)])

# example embedding
example_word = 'bomb'
word_vec = w2v_model.wv[example_word]
print('learned embedding of bomb: ', word_vec)

# !!! YOUR TASK (2 point): show the top-10 words most similar to 'bomb'
# you may calculate manually, or refer to gensim function: https://radimrehurek.com/gensim/models/word2vec.html



# !!! Your TASK (1 point): tune the arguments of Word2Vec to obtain different word embeddings 
# and show top-10 similar words as ''



In [ ]:
# Use basic ML models on word embeddings

# !!! YOUR TASK (2 points): turn word embedding to doc embedding
# Choice 1 (as shown below): manualy take the average of word embeddings in each doc
# Choise 2: gensim doc2vec: https://radimrehurek.com/gensim/models/doc2vec.html 
def doc2vec(docs, w2v_model):
    w2v = []
    for doc in docs:
        word_embs = [w2v_model.wv[w] for w in doc if w in w2v_model.wv] # get a list of word embeddings
        
        # TODO: take (weighted) average to get doc embedding
        doc_emb = np.mean(np.array(word_embs), axis=0) 
        
        # filling zeros for empty doc
        if len(word_embs) == 0:
            doc_emb = [0 for i in range(w2v_model.vector_size)]
        
        w2v.append(doc_emb)
    
    return np.array(w2v)
        
# transform training tweets into embeddings
train_w2v = doc2vec(train_docs, w2v_model)
print('training word2vec shape: ', train_w2v.shape)

test_w2v = doc2vec(test_x['text'], w2v_model) 
print('test word2vec shape: ', test_w2v.shape)


# !!! YOUR TASK (2 point): apply a basic ML model on word embeddings
# you may consider tune the word2vec model to gain better embeddings (refer to gensim document about argument details: https://radimrehurek.com/gensim/models/word2vec.html)
X_train,X_val,y_train,y_val = train_test_split(train_w2v, train['target'], test_size=0.3, random_state=11)



# Optional (+1 bonus pt): Classification directly using RNN model
RNN model can take in word, turn them into embedding and make predictions

Torch tutorial: https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

In [ ]:
# Preprocess text via tokenizer

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")
train_iter = train['text']


def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
print('size of vocab: ', len(vocab))

In [ ]:
# Build a simple RNN model

from torch import nn


# RNN can be a combination of feature extractor (word2vec) and a classifer
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False) # word2vec
        self.fc = nn.Linear(embed_dim, num_class) # classifier
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
# Optional Task (+1 bonus point): follow the tutorial to train RNN on the given dataset


# Save submission file

In [ ]:
y_pred = pd.DataFrame({
    'id': test_x['id'],
    'target': !!! # your prediction
    }
)

y_pred.to_csv("test_submission.csv", index=False)
y_pred.head(3)